In [1]:
import numpy as np

In [2]:
import torch
from torch.utils.data.dataset import Dataset  # For custom data-sets
from torchvision import transforms
from PIL import Image
import glob



In [3]:
import os
import numpy as np

In [24]:
#datagenerator for 90 degree rotation
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform = None):   # initial logic happens like transform

        self.root_dir = root_dir
        image_list = []
        for filename in os.listdir(self.root_dir):
            image_list.append(filename)
        
        self.image_list = image_list
        
    
    def __len__(self):  # return count of sample we have
        return len(self.image_list)
    
    
    def __getitem__(self,index):
        
        image = Image.open(self.root_dir + self.image_list[index])
        image = image.rotate(90)
        if new = 1:
            image = 
        image = np.asarray(image)
 
        return image



In [25]:
custom_img = CustomDataset(path)

In [26]:
train_loader = torch.utils.data.DataLoader(custom_img, batch_size=32, num_workers=1)

In [27]:
im = (iter(train_loader))

/ext3/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return default_collate([torch.as_tensor(b) for b in batch])


In [28]:
image = next(im)

In [29]:
print(image.shape)

torch.Size([32, 96, 96, 3])


In [ ]:
#new day new start

In [117]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, labels, transform = None):   # initial logic happens like transform

        self.root_dir = root_dir
        image_list = []
        
        #normalizing images
        self.img_means = np.array([0.49865985, 0.47058615, 0.41846788])
        self.img_stds = np.array([0.28398806, 0.2754099, 0.29393724])
        
        
        
        self.transform = transforms.Compose([transforms.ToTensor(),
                                             transforms.Normalize(mean=self.img_means, std=self.img_stds)])
        
        
        self.final_images = transforms.Compose([transforms.ToTensor()])
        
        #self.transform = transform
        
        for filename in os.listdir(self.root_dir):
            image_list.append(filename)
        
        self.image_list = image_list
        self.labels = labels
        
    
    def __len__(self):  # return count of sample we have
        return len(self.image_list)
    
    
    def __getitem__(self, index):
        
        image = Image.open(self.root_dir + self.image_list[index])
        label = self.labels[index]
        
        #if self.transform is not None:
         #   image = self.final_image(image)

        
        tensor_image = self.final_images(image)
        #print(tensor_image.shape)
        
        
        #image = image.rotate(90)
        
        
        #image = np.asarray(image)
        
        
        #new change 25 april
        
        #tensor_image = torch.Tensor(image)
        
        
        #tensor_image = tensor_image.permute((2,0,1))
        
        #print(tensor_image.shape)

        output_list = []
        output_list.append(tensor_image)

        #for labels converting to one-hot encode
        ones = torch.sparse.torch.eye(800)
        shape = label.shape
        res = ones.index_select(0, label.view(-1).type(torch.int64))
        final_labels = res.view(*shape, res.shape[-1])


        output_list.append(label)
        output_list.append(final_labels)

        return output_list

In [118]:
tansformers = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize(mean=config.img_means, std=config.img_stds)])
                self.mask_tf = transforms.Compose([transforms.ToTensor()])

IndentationError: unexpected indent (<ipython-input-118-a0ccf90c0fa8>, line 3)

In [119]:
train_path = 'Dataset/final_data/dataset/train/'
valid_path = 'Dataset/final_data/dataset/val/'

In [120]:
train_label_path = 'Dataset/final_data/dataset/train_label_tensor.pt'

In [121]:
validation_labels_path = 'Dataset/final_data/dataset/val_label_tensor.pt'

In [122]:
X = torch.load(train_label_path)

In [123]:
Y = torch.load(validation_labels_path)

In [124]:
dataset = CustomDataset(train_path, X)

In [125]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, num_workers=2)

In [126]:
val_dataset = CustomDataset(valid_path, Y)

In [127]:
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 64, num_workers = 2)

In [128]:
#First simple architecture 

In [129]:
import torch.nn as nn
import torch.nn.functional as F

In [130]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn_block_1 = nn.Sequential(
          nn.Conv2d(3, 128, kernel_size=3, padding=1, stride=1),
          nn.ReLU(inplace=True),
          nn.BatchNorm2d(128),
          nn.MaxPool2d(kernel_size=4, stride=4),

          nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=1),
          nn.ReLU(inplace=True),
          nn.BatchNorm2d(256),
          nn.MaxPool2d(kernel_size=4, stride=4),

        )


        self.cnn_block_2 = nn.Sequential(
          nn.Conv2d(256, 512, kernel_size=3, padding=1, stride=1),
          nn.ReLU(inplace=True),
          nn.BatchNorm2d(512),
          nn.MaxPool2d(kernel_size=2, stride=2))

        self.cnn_block_3 = torch.nn.Sequential(
          nn.Conv2d(32, 64, kernel_size=7, padding=3, stride=1),
          nn.ReLU(inplace=True),
          nn.BatchNorm2d(64),

          nn.Conv2d(64, 32, kernel_size=7, padding=3, stride=1),
          nn.ReLU(inplace=True),
          nn.BatchNorm2d(32),
        )

        self.linear_layers = nn.Linear(4608, 800)
    

    def forward(self, x):
        x = self.cnn_block_1(x)
        x = self.cnn_block_2(x)
        #x = self.cnn_block_2(x) + x
        x = x.view(x.size(0), -1) # flatten
        x = self.linear_layers(x)
        #x = self.linear_layers(x).view(x.size(0))
        #print(x.shape)
        #x = F.log_softmax(x, dim = 1)
        return x

In [131]:
#ResNet models

In [132]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * 8, 1000)
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #print(x.shape)
        x = self.layer4(x)
        #print(x.shape)
        x = self.avgpool(x)
        #print(x.shape)
        x = x.reshape(x.shape[0], -1)
        #print(x.shape)
        x = self.fc(x)
        #print(x.shape)
        x = self.fc2(x)
        

        #x = F.log_softmax(x, dim = 1)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)




In [133]:
#Code for customizing resnets to ResnNet50, 101, and further more

In [134]:

def ResNet50(img_channel=3, num_classes=800):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


def ResNet101(img_channel=3, num_classes=800):
    return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


def ResNet152(img_channel=3, num_classes=800):
    return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)



In [135]:
from tqdm.notebook import tqdm

In [140]:
#training and calculating loss


def get_loss_and_correct(model, batch, criterion, device):
    image = batch[0].to(device)
    label = batch[1].to(device)
    loss_labels = batch[2].to(device)
    result = model(image)
    #print(label.shape)
  #aasdf

    extra, predicted = torch.max(result.data, 1)

  #print(predicted)
  #print(label)
  #print(extra.shape)
  #print(predicted.shape)
    correct_answers = (predicted  == label).sum().item() 

  #print(correct_answers)

    loss = criterion(result, label)

    #loss = loss/len(label)

  #print(loss)

    return loss, correct_answers


def step(loss, optimizer):

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



N_EPOCHS = 50
BATCH_SIZE = 64

model = ResNet152()



criterion = nn.CrossEntropyLoss()



optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)




if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []

pbar = tqdm(range(N_EPOCHS))

for i in pbar:
    total_train_loss = 0.0
    total_train_correct = 0.0
    total_validation_loss = 0.0
    total_validation_correct = 0.0
    
    model.train()
    for batch in tqdm(train_dataloader, leave = False):
        loss, correct = get_loss_and_correct(model, batch, criterion, device)
        step(loss, optimizer)
        total_train_loss += loss.item()
        #print(correct)
        total_train_correct += correct

    with torch.no_grad():
        for batch in validation_dataloader:
            loss, correct = get_loss_and_correct(model, batch, criterion, device)
            total_validation_loss += loss.item()
            total_validation_correct += correct
        
    mean_train_loss = total_train_loss / len(dataset)
    train_accuracy = total_train_correct / len(dataset)

    mean_validation_loss = total_validation_loss / len(dataset)
    validation_accuracy = total_validation_correct / len(dataset)

    train_losses.append(mean_train_loss)
    validation_losses.append(mean_validation_loss)

    train_accuracies.append(train_accuracy)
    validation_accuracies.append(validation_accuracy)

    print({'train_loss': mean_train_loss, 'validation_loss': mean_validation_loss, 'train_accuracy': train_accuracy, 'validation_accuracy': validation_accuracy})
    
    
    
    
    #saving the models
    
    torch.save(model.cpu().state_dict(), 'model.pth')
    model.cuda()
    print('model saved')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

{'train_loss': 0.09696897288784384, 'validation_loss': 0.12313575255684554, 'train_accuracy': 0.004140625, 'validation_accuracy': 0.00125}
model saved


  0%|          | 0/400 [00:00<?, ?it/s]

{'train_loss': 0.09461434472352266, 'validation_loss': 0.13185149971395732, 'train_accuracy': 0.0028515625, 'validation_accuracy': 0.0012109375}
model saved


...

{'train_loss': 0.06411079463548958, 'validation_loss': 0.6039677166286856, 'train_accuracy': 0.0685546875, 'validation_accuracy': 0.001171875}
model saved


  0%|          | 0/400 [00:00<?, ?it/s]

{'train_loss': 0.06372015343979001, 'validation_loss': 0.4830069513618946, 'train_accuracy': 0.0701171875, 'validation_accuracy': 0.0010546875}
model saved


  0%|          | 0/400 [00:00<?, ?it/s]

{'train_loss': 0.06186610935721546, 'validation_loss': 0.39614894294179975, 'train_accuracy': 0.0796875, 'validation_accuracy': 0.00125}
model saved
